# Automatic Command Execution on Boot

This tutorial is aimed at demonstrating how the main control unit automatically executes specific commands and communicates instructions to the subordinate device each time the system boots. The code blocks in this chapter are for comprehension only and are not executable. They serve to elucidate the automatic processes that the product undertakes upon startup. Should you find the need, these commands are subject to modification or expansion.

## cmd_on_boot() Function
The cmd_on_boot() function, located within the main program of the product, defines a list of commands to be executed at startup. These commands facilitate initial configurations and set up essential operational parameters for the device.

In [ ]:
def cmd_on_boot():
    # List of commands to be executed at startup
    cmd_list = [
        'base -c {"T":142,"cmd":50}',   # set feedback interval
        'base -c {"T":131,"cmd":1}',    # serial feedback flow on
        'base -c {"T":143,"cmd":0}',    # serial echo off
        'base -c {{"T":4,"cmd":{}}}'.format(f['base_config']['module_type']),      # select the module - 0:None 1:RoArm-M2-S 2:Gimbal
        'base -c {"T":300,"mode":0,"mac":"EF:EF:EF:EF:EF:EF"}',  # the base won't be ctrl by esp-now broadcast cmd, but it can still recv broadcast megs.
        'send -a -b'    # add broadcast mac addr to peer
    ]
    
    for i in range(0, len(cmd_list)):
        cmdline_ctrl(cmd_list[i])
        cvf.info_update(cmd_list[i], (0,255,255), 0.36)
    # set the mainType(chassis type) and the module type
    set_version(f['base_config']['main_type'], f['base_config']['module_type'])


The control unit of the product can perform certain functional controls via command line instructions, similar to the base -c command shown above. These commands are designed to directly pass JSON instructions written afterwards through the Raspberry Pi's GPIO serial port to the subordinate device. We will further explain the meaning of the default automatic boot-up commands
.
- 'base -c {"T":142,"cmd":50}'
>sets the extra interval time for the subordinate device to continuously feedback information. The unit for the cmd value is milliseconds. This feature is used to reduce the frequency of feedback information from the subordinate device, aiming to alleviate the computational pressure on the control unit from processing this feedback.
。
- 'base -c {"T":131,"cmd":1}'
> turns on the continuous information feedback feature of the subordinate device. Once activated, the control unit does not need to fetch information from the subordinate device in a query-response manner. Although this feature is normally enabled by default on the subordinate device, we send the command again to ensure it's activated.
。
- 'base -c {"T":143,"cmd":0}'
>turns off the serial command echo. This way, when the control unit sends instructions to the subordinate device, the latter will not feedback the received instructions to the control unit, preventing the control unit from processing unnecessary information.
。
- 'base -c {"T":4,"cmd":2}'
>sets the type of the external module. A cmd value of 0 indicates no external module is connected; 1 stands for a robotic arm; and 2 for a gimbal. If your product does not have a gimbal or robotic arm installed, this value should be changed to 0.
。
- 'base -c {"T":300,"mode":0,"mac":"EF:EF:EF:EF:EF:EF"}'
>prevents the chassis from being controlled by ESP-NOW broadcasts from other devices, except for devices with the specified MAC address. You can make up a MAC address or use the MAC address of your own ESP32 remote controller.
。
- 'send -a -b'
>adds the broadcast address (FF:FF:FF:FF:FF:FF) to peers, enabling you to subsequently send broadcast messages directly to other devices via broadcast signals.。

You can learn about other host computer command line instructions in the following WEB command line application chapters.